In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Import Data from CSV
df_test = pd.read_csv('../input/titanic/test.csv')
df_train = pd.read_csv('../input/titanic/train.csv')

#Display Data Test
df_test.head()

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [2]:
#Display Data Train
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#Inspect Data Train
df_train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [4]:
#Clean Data - To be done later after I get access to Lab content again on EdX

    #Count Rows
index = df_train.index
number_of_rows = len(index)
print(number_of_rows)

891


In [5]:
    # Count total NaN at each column in a DataFrame
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df_train.isnull().sum())

 
Count total NaN at each column in a DataFrame : 

 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [6]:
    #Drop 'Cabin' to many NaN's, Drop 'Ticket', 'Name' as not relevant
df_train = df_train.drop(columns=['Cabin', 'Name', 'Ticket'])

    #Fill missing values in 'Age'
df_train['Age'] = df_train['Age'].interpolate()

    #Fill mising Values in 'Embarked'
df_train['Embarked'] = df_train['Embarked'].interpolate()

    # Count total NaN at each column in a DataFrame
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df_train.isnull().sum())

 
Count total NaN at each column in a DataFrame : 

 PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       2
dtype: int64


In [7]:
#Get Corrolation of train data
df_train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.025332,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.062164,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.304934,0.083081,0.018443,-0.549500
Age,0.025332,-0.062164,-0.304934,1.000000,-0.213410,-0.170013,0.087119
SibSp,-0.057527,-0.035322,0.083081,-0.213410,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.170013,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.087119,0.159651,0.216225,1.000000


In [8]:
#Create columns for Pclass, Sex, Embarked to allow analysis and modelling

    #Copy df_train to df_train_2
df_train_2 = df_train.copy()

    #Convert Pclass, Sex, Embarked to columns and attach to df_train_2
dummies = []
cols = ['Pclass','Sex','Embarked']
for col in cols:
    dummies.append(pd.get_dummies(df_train_2[col]))
titanic_dummies = pd.concat(dummies, axis=1)
df_train_2 = pd.concat((df_train_2,titanic_dummies),axis=1)
df_train_2 = df_train_2.drop(['Pclass','Sex','Embarked'],axis=1)
df_train_2.head()

,PassengerId,Survived,Age,SibSp,Parch,Fare,1,2,3,female,male,C,Q,S
0,1,0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1
1,2,1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0
2,3,1,26.0,0,0,7.9250,0,0,1,1,0,0,0,1
3,4,1,35.0,1,0,53.1000,1,0,0,1,0,0,0,1
4,5,0,35.0,0,0,8.0500,0,0,1,0,1,0,0,1


In [9]:
#Create ML

    #Import sklearn
import sklearn
from sklearn.model_selection import train_test_split


    #Create Numpy array
X = df_train_2.values
y = df_train_2['Survived'].values

    #Drop 'Survived' from X
X = np.delete(X,1,axis=1)

    #Split data set to train and test 70/30 split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=0)

    #Create Decision Tree
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=5)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.7910447761194029

In [10]:
from sklearn import ensemble
clf = ensemble.RandomForestClassifier(n_estimators=100)
clf.fit (X_train, y_train)
clf.score (X_test, y_test)

0.8059701492537313

In [11]:
clf = ensemble.GradientBoostingClassifier()
clf.fit (X_train, y_train)
clf.score (X_test, y_test)

0.8134328358208955

In [12]:
clf = ensemble.GradientBoostingClassifier(n_estimators=50)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.832089552238806

In [13]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [14]:
    # Count total NaN at each column in a DataFrame
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df_test.isnull().sum())

 
Count total NaN at each column in a DataFrame : 

 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [15]:
    #Drop 'Cabin' to many NaN's, Drop 'Ticket', 'Name' as not relevant
df_test = df_test.drop(columns=['Cabin', 'Name', 'Ticket'])

    #Fill missing values in 'Age'
df_test['Age'] = df_test['Age'].interpolate()

    #Fill missing values in 'Fare'
df_test['Fare'] = df_test['Fare'].interpolate()

    # Count total NaN at each column in a DataFrame
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df_test.isnull().sum())

 
Count total NaN at each column in a DataFrame : 

 PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64


In [16]:
    #Convert Pclass, Sex, Embarked to columns and attach to df_test
dummies = []
cols = ['Pclass','Sex','Embarked']
    for col in cols:
        dummies.append(pd.get_dummies(df_test[col]))
titanic_dummies = pd.concat(dummies, axis=1)
df_test = pd.concat((df_test,titanic_dummies),axis=1)
df_test = df_test.drop(['Pclass','Sex','Embarked'],axis=1)

df_test.head()

,PassengerId,Age,SibSp,Parch,Fare,1,2,3,female,male,C,Q,S
0,892,34.5,0,0,7.8292,0,0,1,0,1,0,1,0
1,893,47.0,1,0,7.0000,0,0,1,1,0,0,0,1
2,894,62.0,0,0,9.6875,0,1,0,0,1,0,1,0
3,895,27.0,0,0,8.6625,0,0,1,0,1,0,0,1
4,896,22.0,1,1,12.2875,0,0,1,1,0,0,0,1


In [17]:
#Run Model clf on Test Data
X_results = df_test.values
y_results = clf.predict(X_results)

In [18]:
output = np.column_stack((X_results[:,0],y_results))
df_results = pd.DataFrame(output.astype('int'),columns=['PassengerID','Survived'])
df_results.to_csv('titanic_results.csv',index=False)